In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch
# %env CUDA_VISIBLE_DEVICES=1

from lib import data_processors, utils

In [2]:
from lib.bert import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib.train_eval import train, evaluate
from lib.train_student import train_student 
from lib.blend_cnn import BlendCNN
from pytorch_pretrained_bert.modeling import (BertConfig, WEIGHTS_NAME, CONFIG_NAME)
from lib.bert import BertForSequenceClassification

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
params = {
    'data_dir': '../../data/SST-2',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'sst2',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 5,
    'seed': 1331
}

processors = {
    'cola': data_processors.ColaProcessor,
    'mnli': data_processors.MnliProcessor,
    'mrpc': data_processors.MrpcProcessor,
    'sst2': data_processors.SST2Processor
}

num_labels_task = {
    'cola': 2,
    'mnli': 3,
    'mrpc': 2,
    'sst2': 2
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [4]:
processor = processors[params['task_name']]()
num_labels = num_labels_task[params['task_name']]
label_list = processor.get_labels()

tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
eval_examples = processor.get_dev_examples(params['data_dir'])

04/10/2019 20:19:56 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/shakhrayv/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Инициализируем модель

In [5]:
model = BlendCNN(params['max_seq_length'], len(tokenizer.vocab), len(label_list), n_layers=8).to(device)

Код для обучения модели

In [10]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from pytorch_pretrained_bert.optimization import BertAdam
from tqdm import tqdm, trange, tqdm_notebook
from torch.nn import CrossEntropyLoss


def train_blend_cnn(model, train_examples, label_list, params, tokenizer):
    train_steps_per_epoch = int(len(train_examples) / params['train_batch_size'])
    num_train_optimization_steps = train_steps_per_epoch * params['num_train_epochs']

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer
                    if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer
                    if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0}
    ]
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=params['learning_rate'],
                         warmup=params['warmup_proportion'],
                         t_total=num_train_optimization_steps)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0
    
    train_features = utils.convert_examples_to_features(
        train_examples, label_list, params['max_seq_length'], tokenizer)
    
    print("***** Running training *****")
    print("Num examples:",  len(train_examples))
    print("Batch size:  ", params['train_batch_size'])
    print("Num steps:   ", num_train_optimization_steps)
    all_input_ids = torch.tensor([f.input_ids for f in train_features],
                                 dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features],
                                  dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features],
                                   dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in train_features],
                                 dtype=torch.long)
    
    
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data, sampler=train_sampler, batch_size=params['train_batch_size'])

    model.train()
    for epoch_num in range(int(params['num_train_epochs'])):
        print('Epoch: {}'.format(epoch_num + 1))
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            
            logits_model = model(input_ids, segment_ids, input_mask)
            loss = CrossEntropyLoss()(logits_model.view(-1, model.num_labels), label_ids.view(-1))
            loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
    result = {
        'loss': tr_loss / nb_tr_steps,
        'global_step': global_step
    }
    
    return result

Обучаем. Самым лучшим результатом за все время обучения стало 0.62

In [7]:
params['num_train_epochs'] = 1
for i in range(20):
    print('______Epoch_______: {}'.format(i + 1))
    result = train_blend_cnn(model, train_examples, label_list, params, tokenizer)
    result = evaluate(model, eval_examples, label_list, params, tokenizer)
    print(result)
    

______Epoch_______: 1


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 102.61it/s]
04/10/2019 20:20:34 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 20:20:34 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 20:20:34 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 341.77it/s]
04/10/2019 20:20:34 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 20:20:34 - INFO - lib.train_eval -     eval_accuracy = 0.481651376146789
04/10/2019 20:20:34 - INFO - lib.train_eval -     eval_loss = 0.03411867877964033


{'eval_loss': 0.03411867877964033, 'eval_accuracy': 0.481651376146789}
______Epoch_______: 2
***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


...

{'eval_loss': 0.041032327680822905, 'eval_accuracy': 0.6181192660550459}
______Epoch_______: 20


Iteration:   0%|          | 9/2105 [00:00<00:23, 89.75it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 101.08it/s]
04/10/2019 20:29:32 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 20:29:32 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 20:29:32 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 343.14it/s]
04/10/2019 20:29:32 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 20:29:32 - INFO - lib.train_eval -     eval_accuracy = 0.6158256880733946
04/10/2019 20:29:32 - INFO - lib.train_eval -     eval_loss = 0.038944251860978954


{'eval_loss': 0.038944251860978954, 'eval_accuracy': 0.6158256880733946}


In [8]:
for i in range(20, 40):
    print('______Epoch_______: {}'.format(i + 1))
    result = train_blend_cnn(model, train_examples, label_list, params, tokenizer)
    result = evaluate(model, eval_examples, label_list, params, tokenizer)
    print(result)
    

______Epoch_______: 21


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 101.82it/s]
04/10/2019 20:54:59 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 20:54:59 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 20:54:59 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 333.18it/s]
04/10/2019 20:54:59 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 20:54:59 - INFO - lib.train_eval -     eval_accuracy = 0.6204128440366973
04/10/2019 20:54:59 - INFO - lib.train_eval -     eval_loss = 0.04366837294960241


{'eval_loss': 0.04366837294960241, 'eval_accuracy': 0.6204128440366973}
______Epoch_______: 22


...

{'eval_loss': 0.13462375876826976, 'eval_accuracy': 0.6158256880733946}
______Epoch_______: 40


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 100.74it/s]
04/10/2019 21:04:03 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 21:04:03 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 21:04:03 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 331.59it/s]
04/10/2019 21:04:03 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 21:04:03 - INFO - lib.train_eval -     eval_accuracy = 0.6123853211009175
04/10/2019 21:04:03 - INFO - lib.train_eval -     eval_loss = 0.14147627627084014


{'eval_loss': 0.14147627627084014, 'eval_accuracy': 0.6123853211009175}


In [9]:
for i in range(40, 60):
    print('______Epoch_______: {}'.format(i + 1))
    result = train_blend_cnn(model, train_examples, label_list, params, tokenizer)
    result = evaluate(model, eval_examples, label_list, params, tokenizer)
    print(result)
    

______Epoch_______: 41


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:21<00:00, 102.77it/s]
04/10/2019 21:20:51 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 21:20:51 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 21:20:51 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 341.67it/s]
04/10/2019 21:20:52 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 21:20:52 - INFO - lib.train_eval -     eval_accuracy = 0.6318807339449541
04/10/2019 21:20:52 - INFO - lib.train_eval -     eval_loss = 0.13185403656658776


{'eval_loss': 0.13185403656658776, 'eval_accuracy': 0.6318807339449541}
______Epoch_______: 42


...

{'eval_loss': -0.005754406213623668, 'eval_accuracy': 0.6135321100917431}
______Epoch_______: 60


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104
Epoch: 1


Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 100.48it/s]
04/10/2019 21:29:54 - INFO - lib.train_eval -   ***** Running evaluation *****
04/10/2019 21:29:54 - INFO - lib.train_eval -     Num examples = 872
04/10/2019 21:29:54 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 342.96it/s]
04/10/2019 21:29:54 - INFO - lib.train_eval -   ***** Eval results *****
04/10/2019 21:29:54 - INFO - lib.train_eval -     eval_accuracy = 0.6181192660550459
04/10/2019 21:29:54 - INFO - lib.train_eval -     eval_loss = 0.026595231794982875


{'eval_loss': 0.026595231794982875, 'eval_accuracy': 0.6181192660550459}
